In [4]:
import os
import numpy as np
import librosa
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score

# ==========================================
# 1. KONFIGURACJA I ŁADOWANIE MODELU AI
# ==========================================

# VGGish wymaga sygnału 16kHz
SR_VGG = 16000
VGGISH_MODEL_URL = "https://tfhub.dev/google/vggish/1"

print("⏳ Ładowanie modelu VGGish z TensorFlow Hub (może chwilę potrwać)...")
vggish_model = hub.load(VGGISH_MODEL_URL)
print("✅ Model VGGish gotowy.")

def preprocess_audio_for_vggish(y, sr):
    """
    Zamienia sygnał audio na format akceptowany przez VGGish.
    Zwraca tensor o kształcie (Batch, 96, 64).
    """
    # 1. Resampling do 16kHz (wymóg VGGish)
    if sr != SR_VGG:
        y = librosa.resample(y, orig_sr=sr, target_sr=SR_VGG)
    
    # 2. Normalizacja
    y = y / np.max(np.abs(y)) if np.max(np.abs(y)) > 0 else y

    # 3. Tworzenie spektrogramu Log Mel (zgodnie z parametrami VGGish)
    stfts = librosa.stft(y, n_fft=400, hop_length=160, win_length=400)
    spectrogram = np.abs(stfts)
    mel_spectrogram = librosa.feature.melspectrogram(
        S=spectrogram, sr=SR_VGG, n_mels=64, fmin=125, fmax=7500
    )
    log_mel_spectrogram = np.log(mel_spectrogram + 0.01).T # (Czas, Cechy)
    
    # 4. Formatowanie do wejścia sieci (klatki po 0.96s)
    # VGGish oczekuje wejścia [num_examples, 96, 64]
    num_frames = 96  # 0.96s
    total_frames = log_mel_spectrogram.shape[0]
    
    # Jeśli plik jest krótszy niż 0.96s, robimy padding (dopełnienie zerami)
    if total_frames < num_frames:
        pad_width = num_frames - total_frames
        features_padded = np.pad(log_mel_spectrogram, ((0, pad_width), (0, 0)), mode='constant')
        return np.expand_dims(features_padded, axis=0)
    
    # Jeśli dłuższy, bierzemy tyle pełnych segmentów, ile się zmieści
    num_segments = total_frames // num_frames
    features_truncated = log_mel_spectrogram[:num_segments * num_frames]
    return features_truncated.reshape((num_segments, num_frames, 64))

# ==========================================
# 2. ŁADOWANIE DANYCH (Poprawiona ścieżka)
# ==========================================

# UWAGA: Tutaj zmiana na podstawie Twojego pliku tree.txt
DATA_DIR = '../../dataset/' 

vgg_features = []
labels = []

print(f"\n📂 Szukam danych w: {os.path.abspath(DATA_DIR)}")

if not os.path.exists(DATA_DIR):
    print(f"❌ BŁĄD: Katalog '{DATA_DIR}' nie istnieje. Sprawdź, gdzie znajduje się Twój plik .ipynb.")
else:
    classes = [d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))]
    print(f"Znalezione klasy: {classes}")

    for label in classes:
        class_dir = os.path.join(DATA_DIR, label)
        file_list = [f for f in os.listdir(class_dir) if f.lower().endswith('.wav')]
        
        print(f"   -> Przetwarzanie '{label}': {len(file_list)} plików...")
        
        for filename in file_list:
            file_path = os.path.join(class_dir, filename)
            try:
                # Wczytanie
                y, sr = librosa.load(file_path, sr=None)
                
                # Preprocessing
                vgg_input = preprocess_audio_for_vggish(y, sr)
                
                # Inferencja (Sieć neuronowa "słucha" pliku)
                embeddings = vggish_model(vgg_input)
                
                # Uśrednienie wyników dla całego pliku (1 plik = 1 wektor cech)
                file_embedding = np.mean(embeddings.numpy(), axis=0)
                
                vgg_features.append(file_embedding)
                labels.append(label)
                
            except Exception as e:
                print(f"⚠️ Błąd przy pliku {filename}: {e}")

    # Konwersja do numpy array
    X = np.array(vgg_features)
    y_labels = np.array(labels)

    print(f"\n✅ Zakończono! Wymiary macierzy cech: {X.shape}")
    # Oczekiwany kształt: (Liczba_plików, 128)

# ==========================================
# 3. TRENING I OCENA (Klasyfikator SVM)
# ==========================================

if len(X) > 0:
    # Encodowanie etykiet (tekst -> liczby)
    le = LabelEncoder()
    y_enc = le.fit_transform(y_labels)

    # Podział na zbiór treningowy i testowy
    X_train, X_test, y_train, y_test = train_test_split(
        X, y_enc, test_size=0.3, random_state=42, stratify=y_enc
    )

    # Pipeline: Standaryzacja + SVM (RBF kernel)
    # To samo, co w podejściu klasycznym
    clf = make_pipeline(
        StandardScaler(),
        SVC(kernel='rbf', probability=True, random_state=42)
    )

    # Trenowanie
    print("\n🚀 Trenowanie klasyfikatora SVM na cechach z VGGish...")
    clf.fit(X_train, y_train)

    # Predykcja
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)

    print(f"\n🏆 WYNIK PODEJŚCIA 'AI' (VGGish + SVM):")
    print(f"Dokładność (Accuracy): {acc:.2%}")

    # Macierz pomyłek
    plt.figure(figsize=(8, 6))
    sns.heatmap(confusion_matrix(y_test, y_pred), annot=True, fmt='d', cmap='Blues',
                xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title(f'Macierz Pomyłek (VGGish Embeddings)\nAcc: {acc:.2%}')
    plt.ylabel('Prawdziwa klasa')
    plt.xlabel('Przewidziana klasa')
    plt.show()

else:
    print("❌ Nie udało się stworzyć zbioru danych. Sprawdź ścieżki.")

⏳ Ładowanie modelu VGGish z TensorFlow Hub (może chwilę potrwać)...
✅ Model VGGish gotowy.

📂 Szukam danych w: /home/muzykos/Documents/Github/Projekt_badawczy_System_rozpoznawania_zdarzen_dzwiekowych_w_przestrzeni_domowej/dataset
Znalezione klasy: ['footsteps', 'doorbell', 'door_knock', 'faucet', 'clapping', 'hairdryer', 'vacuum_cleaner']
   -> Przetwarzanie 'footsteps': 100 plików...
⚠️ Błąd przy pliku footsteps_089.wav: Binding inputs to tf.function failed due to `Can not cast TensorSpec(shape=(1, 96, 64), dtype=tf.float32, name=None) to TensorSpec(shape=(None,), dtype=tf.float32, name=None)`. Received args: (array([[[-4.2390227, -4.193543 , -4.231896 , ..., -4.5802054,
         -4.5816164, -4.5885944],
        [-4.150315 , -4.2256365, -4.4522314, ..., -4.573516 ,
         -4.571542 , -4.5803776],
        [-4.2748275, -4.284068 , -4.3695946, ..., -4.575117 ,
         -4.5667806, -4.5671425],
        ...,
        [-4.0184135, -3.3499205, -3.4195225, ..., -4.5480304,
         -4.560118